In [ ]:
from sys import path

import numpy as np
import pandas as pd 
from scipy.stats import norm
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_theme()

%load_ext autoreload
%autoreload 2

# user-written 
import w8_estimation as est 
import w8_LinearModel as lm
import w8_probit as probit
import w8_logit as logit
import marginal_effects as me


In [ ]:
# Outcome label
y_lab = 'anyuseofforce_coded'

# Dataset columns
rawdat_columns = [
    'anyuseofforce_coded',
    
    # Subject (civilian) characteristics
    'sblack',
    'shisp',
    'swhite',
    'smale',
    'sother', 
    'sage',
    'sempl', 
    'sincome',
    'spop', 
    'sbehavior',
    
    # Officer characteristics
    'omajblack',
    'omajhisp',
    'omajwhite',
    'omajother', 
    'osplit', 
    
    # Encounter characteristics
    'daytime',
    'inctype_lin', 
    'year'
]


In [ ]:
# Final X-matrix variable labels 
x_lab = [
    # Subject vars (white is reference)
    'sblack',
    'shisp',
    'smale',
    'sage',
    'sincome',

    # Officer vars (white is reference)
    #'omajblack', --> Udelukkes fordi der ikke er nogen
    'omajhisp',
    #'omajother',

    # Encounter vars
    'daytime'
]


In [ ]:
dat = pd.read_csv('ppcs_cc.csv')

N = dat.shape[0]

# reorder columns 
dat = dat[[y_lab] + x_lab].copy()

dat.head(5)

assert dat.notnull().all(axis=1).all(), 'Missings in the dataset, take them out!'

In [ ]:
y = dat[y_lab].values
x = dat[x_lab].values
K = x.shape[1]

print("Shape x:", x.shape)
print("Rank x:", np.linalg.matrix_rank(x))
y.shape

In [ ]:
# OLS estimates
ols_results = lm.estimate(y, x, robust_se=True)
ols_tab = lm.print_table((y_lab, x_lab), ols_results, title='LPM results')
ols_tab

In [ ]:
theta0 = probit.starting_values(y, x)
theta0.ndim==1

In [ ]:
ll = probit.loglikelihood(theta0, y, x)
np.isclose(np.mean(ll), -1.0411283428047824)

In [ ]:
#probit_results = est.estimate(probit.q, theta0, y, x)

In [ ]:
#probit_tab = est.print_table(x_lab, probit_results, title=f'Probit, y = {y_lab}')
#probit_tab

In [ ]:
theta0 = logit.starting_values(y, x)
theta0 

In [ ]:
ll = logit.loglikelihood(theta0, y, x)
print("Logit log-likelihood at starting values:", ll)

In [ ]:
logit_results = est.estimate(logit.q, theta0, y, x)

In [ ]:
logit_tab = est.print_table(x_lab, logit_results, title=f'Logit, y = {y_lab}')
logit_tab

In [ ]:
# Reference covariate profile used for marginal effects
# sincome takes values 1 (low) - 3 (high); use 2 as the midpoint reference
x_ref = np.array([0, 0, 1, 25, 2, 1, 1], dtype=float)
pd.DataFrame(x_ref.reshape(1, -1), columns=x_lab, index=['x_ref'])


In [ ]:
# Delta-method partial effects for logit across all regressors
b_lg = logit_results['theta']
cov_lg = logit_results['cov']

binary_vars = ['sblack', 'shisp', 'smale', 'omajhisp', 'daytime']
continuous_vars = ['sage', 'sincome']

me_rows = []

for var in binary_vars:
    idx = x_lab.index(var)
    x0 = x_ref.copy()
    x1 = x_ref.copy()
    x0[idx] = 0.0
    x1[idx] = 1.0
    effect, se = me.discrete_effect_delta(b_lg, cov_lg, x0, x1, logit.G)
    me_rows.append({
        'Var': var,
        'Effect (LPM)': ols_results['b_hat'][idx],
        's.e. (LPM)': ols_results['se'][idx],
        'Effect (Logit)': effect,
        's.e. (Logit)': se,
    })

for var in continuous_vars:
    idx = x_lab.index(var)
    effect, se = me.continuous_effect_delta(b_lg, cov_lg, x_ref, idx, logit.Gprime)
    me_rows.append({
        'Var': var,
        'Effect (LPM)': ols_results['b_hat'][idx],
        's.e. (LPM)': ols_results['se'][idx],
        'Effect (Logit)': effect,
        's.e. (Logit)': se,
    })

me_tab = pd.DataFrame(me_rows).set_index('Var')
me_tab['t (LPM)'] = me_tab['Effect (LPM)'] / me_tab['s.e. (LPM)']
me_tab['t (Logit)'] = me_tab['Effect (Logit)'] / me_tab['s.e. (Logit)']


In [ ]:
me_tab.round(4)
